In [40]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [41]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data"
ampl.read(mod_path)

# Assign set data

In [42]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [43]:
intervals = pd.read_csv(os.path.join(dataDirectory, "intervals.csv"),skipinitialspace=True)
dffr = pd.read_csv(os.path.join(dataDirectory, "dffr.csv"),skipinitialspace=True)
da = pd.read_csv(os.path.join(dataDirectory, "da.csv"),skipinitialspace=True)

In [44]:
assign_set_data("INTERVALS",intervals.values[:,0])
assign_set_data("DFFR_PRICE",dffr.DFFR_PRICE.values)
assign_set_data("DA_PRICE",da.DA_PRICE.unique())

# Assign parameter data

In [45]:
ampl.getParameter('Cost').set(0)
ampl.getParameter('Ramp').set(999999)
ampl.getParameter('Ramp_DFFR').set(9999999)
ampl.getParameter('P_MAX').set(2)

In [46]:
n_DFFR_PRICE = len(dffr.DFFR_PRICE.unique())

df = pd.DataFrame([1/n_DFFR_PRICE for i in range(n_DFFR_PRICE)],
             columns=["p_R"],
             index = dffr.DFFR_PRICE.values)
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

n_DA_PRICE = len(da.DA_PRICE.unique())
df = pd.DataFrame([1/n_DA_PRICE for i in range(n_DA_PRICE)],
             columns=["p_DA"],
             index = da.DA_PRICE.unique())
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)



In [47]:
df = dffr.set_index("DFFR_PRICE")
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

df = da.set_index(["DA_PRICE","INTERVALS"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)


# Solve the model

In [48]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
knitro_options = 'outlev = 1 mip_outlevel = 1'
solver = 'cplexamp'
settings = {'solver': solver, 'knitro_options':knitro_options,'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [49]:
ampl.solve()

CPLEX 12.5.1.0: mipdisplay = 2
mipgap = .005
Found incumbent of value 0.000000 after 0.00 sec. (0.03 ticks)
MIP Presolve eliminated 396 rows and 321 columns.
All rows and columns eliminated.

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.64 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.64 ticks)
CPLEX 12.5.1.0: optimal integer solution; objective 2004.01
0 MIP simplex iterations
0 branch-and-bound nodes
Tried aggregator 2 times


In [50]:
# Example of getting solution
d_R = ampl.getVariable("d_R").getValues().toPandas()
d_DA = ampl.getVariable("d_DA").getValues().toPandas()



print("Bid at level accepted in this DFFR scenario (and above):")
print(d_R.loc[d_R['d_R.val']==1])

print("Given DFFR scenario i, bid at level accepted in this DA scenario (and above) in time period t:")
print(d_DA.loc[d_DA['d_DA.val']==1])


Bid at level accepted in this DFFR scenario (and above):
     d_R.val
1.0      1.0
Given DFFR scenario i, bid at level accepted in this DA scenario (and above) in time period t:
                 d_DA.val
(1.0, 1.0, 1.0)       1.0
(1.0, 1.0, 2.0)       1.0
(1.0, 1.0, 3.0)       1.0
(1.0, 1.0, 4.0)       1.0
(1.0, 1.0, 5.0)       1.0
(1.0, 1.0, 6.0)       1.0
(2.0, 2.0, 1.0)       1.0
(2.0, 2.0, 2.0)       1.0
(2.0, 2.0, 3.0)       1.0
(2.0, 2.0, 4.0)       1.0
(2.0, 2.0, 5.0)       1.0
(2.0, 2.0, 6.0)       1.0
(3.0, 3.0, 1.0)       1.0
(3.0, 3.0, 2.0)       1.0
(3.0, 3.0, 3.0)       1.0
(3.0, 3.0, 4.0)       1.0
(3.0, 3.0, 5.0)       1.0
(3.0, 3.0, 6.0)       1.0
(4.0, 4.0, 1.0)       1.0
(4.0, 4.0, 2.0)       1.0
(4.0, 4.0, 3.0)       1.0
(4.0, 4.0, 4.0)       1.0
(4.0, 4.0, 5.0)       1.0
(4.0, 4.0, 6.0)       1.0


In [67]:
q_R = ampl.getVariable("q_R").getValues().toPandas()
print("Bid this quantity of DFFR response:")
print(q_R)


q_DA = ampl.getVariable("q_DA").getValues().toPandas()
print("For the given DFFR price scenario and time and Bid this quantity of DFFR response:")
print(q_DA)


Bid this quantity of DFFR response:
   q_R.val
0      2.0
For the given DFFR price scenario and time and Bid this quantity of DFFR response:
            q_DA.val
(1.0, 1.0)       2.0
(1.0, 2.0)       2.0
(1.0, 3.0)       2.0
(1.0, 4.0)       2.0
(1.0, 5.0)       2.0
(1.0, 6.0)       2.0
(2.0, 1.0)       2.0
(2.0, 2.0)       2.0
(2.0, 3.0)       2.0
(2.0, 4.0)       2.0
(2.0, 5.0)       2.0
(2.0, 6.0)       2.0
(3.0, 1.0)       2.0
(3.0, 2.0)       2.0
(3.0, 3.0)       2.0
(3.0, 4.0)       2.0
(3.0, 5.0)       2.0
(3.0, 6.0)       2.0
(4.0, 1.0)       2.0
(4.0, 2.0)       2.0
(4.0, 3.0)       2.0
(4.0, 4.0)       2.0
(4.0, 5.0)       2.0
(4.0, 6.0)       2.0


In [66]:
Q_R = ampl.getVariable("Q_R").getValues().toPandas()
print("Accepted DFFR response in DFFR scenario")
print(Q_R)


Q_DA = ampl.getVariable("Q_DA").getValues().toPandas()
print("For the given DFFR price scenario and time and Bid this quantity of DFFR response:")

print(Q_DA.loc[Q_DA['Q_DA.val']!=0])


Bid this quantity of DFFR response:
     Q_R.val
1.0      0.0
2.0      0.0
3.0      0.0
4.0      0.0
For the given DFFR price scenario and time and Bid this quantity of DFFR response:
                 Q_DA.val
(1.0, 4.0, 1.0)       2.0
(1.0, 4.0, 2.0)       2.0
(1.0, 4.0, 3.0)       2.0
(1.0, 4.0, 4.0)       2.0
(1.0, 4.0, 5.0)       2.0
(1.0, 4.0, 6.0)       2.0
